In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow import keras
from keras.applications.resnet import preprocess_input
from keras.applications.resnet import decode_predictions
from keras.applications.resnet import ResNet50
from keras.applications import InceptionV3

In [ ]:
def append_ext(fn):
    return fn+".png"
def hotOne(fn):
    return fn+".png"
def toString(fn):
    return str(fn)

df = pd.read_csv('../input/vcomlabels/multilabel_small.csv')
df["id"]=df["id"].apply(append_ext)
df["attribute_ids"] = df["attribute_ids"].apply(toString)

df

In [ ]:
X = df['id']
X

In [ ]:
y = df['attribute_ids']
y

In [ ]:
test_percentage = 0.1
test_index = int(len(df) * (1 - test_percentage))
test_index

In [ ]:
c = 0
for index in df.index:
    values = df.loc[index,'attribute_ids'].split()
    y = np.array(len(values))
    if c % 10000 == 0:
        print(c)
    c = c + 1
    df.loc[index, 'attribute_ids'] = set(values)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df2 = pd.DataFrame(mlb.fit_transform(df['attribute_ids']),columns=mlb.classes_)
df2 = pd.concat([df2, df], axis=1)
df2

In [ ]:
dataframe_small = df2.drop(['attribute_ids'], axis = 1)

labels = list(dataframe_small.drop(['id'], axis = 1).keys())
labels

In [ ]:
def process(image):
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

datagen=ImageDataGenerator(rescale=1./255, validation_split=0.2, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                          )

train_generator=datagen.flow_from_dataframe(dataframe=df2, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col=labels, class_mode="raw",
                                            target_size=(224,224), batch_size=16,subset="training")

In [ ]:
validation_generator=datagen.flow_from_dataframe(dataframe=df2, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col=labels, class_mode="raw",
                                            target_size=(224,224), batch_size=16,subset="validation")

In [ ]:
datagen_test=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0,
                            height_shift_range=0,
                            zoom_range=0,
                            horizontal_flip=False,
                          )
test_generator=datagen_test.flow_from_dataframe(dataframe=df2[test_index:], directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col=labels, class_mode="raw",
                                            target_size=(224,224), batch_size=16,subset="training")
                                            

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


In [ ]:
METRICS = [
      keras.metrics.CategoricalAccuracy(name='acc'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      'accuracy',
]


In [ ]:
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
model2 = ResNet50(include_top = False, weights = "imagenet", input_shape=(224, 224, 3))
#model2 = InceptionV3(include_top = False, weights = "imagenet", input_shape=(224, 224, 3))

In [ ]:
flat1 = Flatten()(model2.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
dropout1 = Dropout(0.5)(class1)
output = Dense(28, activation='sigmoid')(dropout1)
# define new model
model2 = Model(inputs=model2.inputs, outputs=output)
model2.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [ ]:
model2.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="binary_crossentropy", metrics=METRICS)
history = model2.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5)

In [ ]:
precision = history.history['val_precision'][4]
recall = history.history['val_recall'][4]

f2beta_score_val = 5 * (precision * recall) / (5 * precision + recall)
f2beta_score_val